In [1]:
# Importing dependencies
import numpy as np
import torch
import tensorflow as tf
import json
import time
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import adam
from torch.optim import sgd
from google.colab import drive
import requests
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

Data getting part

In [2]:
#Mounting drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Loading Data
crypto = "ADA"

try:
    with open("/content/drive/MyDrive/Databases/database_"+crypto+"_T.json", "r") as file:
        jr = json.load(file)
except Exception as e:
    print("An error occured:", e)

def get_last_it(json_file):
    iter = 0
    for i in json_file:
        if int(i) > iter:
            iter = int(i)
    return iter+1

def get_first_it(json_file):
    iter = 0
    for i in json_file:
        if int(i) < iter:
            iter = int(i)
    return iter

last_it = get_last_it(jr)

# Initializing some variables
X = []
y_true = []


# Getting the data organized
for i, details in jr.items():
  try:
    prices = jr[i]["Prices"]
    volume = jr[i]["Volume"]
    Google_Trends_Data = jr[i]["Google Trends"]
    RSI = jr[i]["RSI"]
    Market_cap = jr[i]["Market_cap"]
    market_cap_TVL = jr[i]["MC/TVL"]
    outputs = jr[i]["Price_12_hours"]
    inputs_a = prices + volume + RSI + Google_Trends_Data + Market_cap + market_cap_TVL
  except Exception as e:
    inputs_a = jr[i]["Data"]
  if len(inputs_a) == 207:
    X.append(inputs_a)
    if outputs > 0.01: #mudar dependendo da crypto
        y_true.append([1, 0])
    else:
        y_true.append([0, 1])


def One_Hot_Encoded(y_true):
  y = []
  for i in y_true:
    y.append(np.argmax(i))
  return y

print(len(X))

# [0, 1] --> Sell, [1, 0] --> Buy, Indexes: 1 --> Sell, 0 --> Buy


Algumas funções e classes para a rede neuronal

In [12]:
#Creating Class with pytorch

class Model(nn.Module):
  #Input Layer (=length of inputs) --> Hidden Layer (x5) --> Output layer(Buy or sell)
  def __init__(self, in_features=207, h1=140, h2=100, h3=80, h4=50, h5=30, h6=16, h7=5, ol=2, drop_prob=0.2):
    super().__init__()

    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.fc3 = nn.Linear(h2, h3)
    self.fc4 = nn.Linear(h3, h4)
    self.fc5 = nn.Linear(h4, h5)
    self.fc6 = nn.Linear(h5, h6)
    self.fc7 = nn.Linear(h6, h7)
    self.out = nn.Linear(h7, ol)


  def forward(self, X):
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = F.relu(self.fc3(X))
    X = F.relu(self.fc4(X))
    X = F.relu(self.fc5(X))
    X = F.relu(self.fc6(X))
    X = F.relu(self.fc7(X))
    X = self.out(X)

    return X

torch.manual_seed(0.01)
model = Model()

#Accuracy Function
def accuracy(y_true, y_pred):
  sum = 0
  for i, j in zip(y_true, y_pred):
    if np.argmax(i) == np.argmax(j):
      sum += 1
  return (sum / len(y_true))


Training and testing split

In [5]:
def Baralha_Listas(lista1, lista2, maximo_len=None):
    indexes = []
    for a, i in enumerate(lista1):
        indexes.append(a)
    if maximo_len is None:
        maximo_len = min(len(lista1), len(lista2))
    else:
        maximo_len = min(maximo_len, min(len(lista1), len(lista2)))
    lista1_baralhada = []
    lista2_baralhada = []
    lista_indexes = set()
    while len(lista1_baralhada) < maximo_len and len(lista2_baralhada) < maximo_len:
        random_index2 = random.randint(0, len(indexes)-1)
        random_index = indexes[random_index2]
        if random_index not in lista_indexes:  # Verificar se o índice já foi utilizado
            lista_indexes.add(random_index)
            lista1_baralhada.append(lista1[random_index])
            lista2_baralhada.append(lista2[random_index])
        del indexes[random_index2]
    return lista1_baralhada, lista2_baralhada
def test_train(X, y, test_size=0.2):
    X2_train = []
    y2_train = []
    X2_test = []
    y2_test = []
    le = len(X)
    len_per_train = round((1-test_size)*le)
    len_per_test = int(le - len_per_train)
    X_bar, Y_bar = Baralha_Listas(X, y)
    for i in range(len_per_train):
        y2_train.append(Y_bar[i])
        X2_train.append(X_bar[i])
    for j in range(len_per_test):
        X2_test.append(X_bar[j+len_per_train])
        y2_test.append(Y_bar[j+len_per_train])
    return X2_train, X2_test, y2_train, y2_test

X_train2, X_test2, y_train2, y_test2 = test_train(X, y_true, test_size = 0.1)
X_train = torch.FloatTensor(X_train2)
X_test = torch.FloatTensor(X_test2)
y_train = torch.FloatTensor(y_train2)
y_test = torch.FloatTensor(y_test2)

Putting some relevant variables in

In [18]:
learning_rate = 1e-4
iterations = 1000000

Loss Function and Optimizer

In [19]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Starting to train

In [ ]:
model.train()
loss_opt = []
using = 1
for i in range(iterations):
  y_pred = model.forward(X_train)

  loss = criterion(y_pred, y_train)

  loss_opt.append(loss.detach().numpy())

  if i % 2500 == 0:
    torch.save(model.state_dict(), "/content/drive/MyDrive/Databases/Databases Weights And Biases/"+crypto+"_crypto_predict_AI.pt")
    print("Saved")

  if i % 10 == 0:
    print("Epoch:", i, "Loss:", loss.detach())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  using += 1


Watching it grow

In [ ]:
if using == len(loss_opt)-1:
  using += 1
if using == len(loss_opt)+1:
  using -= 1
plt.plot(range(using), loss_opt)
plt.show()

Evaluate Model

In [21]:
correct = 0
model.eval()
with torch.no_grad():
  y_val = model.forward(X_test)
  loss = criterion(y_val, y_test)

#Accuracy
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_eval = model.forward(data)
    if y_eval.argmax().item() == y_test[i].argmax():
      correct += 1

    #print(i, str(y_val.argmax()))
for i, j in zip(y_val, y_test):
  print("Prediction:", i.argmax())
  print("Real:", j.argmax())



print("Dados Certos:", correct)
print("Numero de dados:", len(y_test))

print("Accuracy:", correct / len(y_test))
print("Loss:", loss)


Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(1)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(0)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: tensor(0)
Real: tensor(1)
Prediction: 

To training continuom

In [9]:
obj = {
    "X_TR": X_train2,
    "X_TT": X_test2,
    "Y_TR": y_train2,
    "Y_TT": y_test2
}
with open("/content/drive/MyDrive/Databases/Databases Weights And Biases/Training_Continuom/continuom_inputs_"+crypto+".json", "w") as file:
  json.dump(obj, file, indent=2)

Inserting information

In [ ]:
new_data = torch.tensor()
prediction = (model.forward(new_data)).argmax()
if prediction == 0:
  print("Buy")
if prediction == 1:
  print("Sell")
print(prediction)

Saving model

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Databases/Databases Weights And Biases/"+crypto+"_crypto_predict_AI.pt")

Saving model and loading module

In [ ]:
new_model = Model()
new_model.load_state_dict(torch.load("/content/drive/MyDrive/Databases/Databases Weights And Biases/"+crypto+"_crypto_predict_AI.pt"))

#Checking if deployed correctly
new_model.eval()

If restarting

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Databases/Databases Weights And Biases/"+crypto+"_crypto_predict_AI.pt"))
with open("/content/drive/MyDrive/Databases/Databases Weights And Biases/Training_Continuom/continuom_inputs_"+crypto+".json", "r") as file:
  data = json.load(file)
X_train = torch.Tensor(data["X_TR"])
X_test = torch.Tensor(data["X_TT"])
y_train = torch.Tensor(data["Y_TR"])
y_test = torch.Tensor(data["Y_TT"])

In [ ]:
model = Model()
model.load_state_dict(torch.load("/content/drive/MyDrive/Databases/Databases Weights And Biases/"+crypto+"_crypto_predict_AI.pt"))

In [ ]:
print(crypto)

SOL


In [ ]:
crypto = "ADA"

try:
    with open("/content/drive/MyDrive/Databases/database_"+crypto+"_T.json", "r") as file:
        jr = json.load(file)
except Exception as e:
    print("An error occured:", e)

def get_last_it(json_file):
    iter = 0
    for i in json_file:
        if int(i) > iter:
            iter = int(i)
    return iter+1

last_it = get_last_it(jr)
first_it = get_first_it(jr)

# Initializing some variables
X = []
y_true = []


# Getting the data organized
for i, details in jr.items():
  try:
    prices = jr[i]["Prices"]
    volume = jr[i]["Volume"]
    Google_Trends_Data = jr[i]["Google Trends"]
    RSI = jr[i]["RSI"]
    Market_cap = jr[i]["Market_cap"]
    market_cap_TVL = jr[i]["MC/TVL"]
    outputs = jr[i]["Price_12_hours"]
    inputs_a = prices + volume + RSI + Google_Trends_Data + Market_cap + market_cap_TVL
  except Exception as e:
    inputs_a = jr[i]["Data"]
  if len(inputs_a) == 207:
    X.append(inputs_a)
    if outputs > 0.0001: #mudar dependendo da crypto
        y_true.append([1, 0])
    else:
        y_true.append([0, 1])

with open("/content/drive/MyDrive/Databases/database_"+crypto+"_T.json", "w") as file:
        json.dump(jr, file, indent=2)